# Optimización de Consultas

## Recordando


Vimos cómo el concepto de una junta o `JOIN` puede entenderse como un **producto cartesiano** entre dos relaciones, seguido de un filtrado mediante el **predicado de la junta** para descartar filas irrelevantes.

Sin embargo, esto no es lo que ocurre en la práctica: calcular el producto cartesiano completo implicaría procesar NxM filas, lo cual resulta ineficiente, ya que la mayoría de esas combinaciones no aportan valor. Por eso, los sistemas de bases de datos emplean estrategias más **optimizadas** para evaluar los `JOIN`, evitando generar combinaciones innecesarias.


>**NOTA:**
>
> Los **índices** en bases de datos son estructuras auxiliares (como árboles o tablas hash) que permiten **acceder rápidamente** a filas según los valores de una o más columnas, evitando recorrer toda la tabla. Funcionan de manera similar al índice de un libro.


## Componentes de Ejecución en un Motor de Base de Datos





El proceso de ejecución de una consulta SQL pasa por varias etapas clave:

1. **Intérprete**: Traduce la consulta `SELECT` escrita en SQL a una **expresión del Álgebra Relacional**. Esta primera forma suele ser **ineficiente**, ya que refleja literalmente la estructura de la consulta.

2. **Optimizador**: Toma esa expresión ineficiente y, basándose en **reglas de transformación** (que veremos más adelante), genera una **expresión equivalente pero más eficiente** para ejecutar.

3. **Generador de código**: Convierte la expresión optimizada en un plan de ejecución físico, es decir, en un **programa eficiente** que accede a los datos reales de forma óptima.

> 🔎 El **optimizador** es un componente central del motor de base de datos: su función es elegir la forma más rápida de obtener los datos pedidos, sin alterar el resultado final.

## Optimización basada en Reglas

### Reglas Generales de Optimización en el Motor de Base de Datos




El optimizador aplica un conjunto de transformaciones sobre la expresión del Álgebra Relacional con el fin de mejorar el rendimiento sin alterar el resultado. Algunas reglas clave son:

- **Empujar predicados de selección** lo más profundo posible en la expresión: permite **filtrar filas tempranamente**, reduciendo el volumen de datos a procesar en etapas posteriores.

- **Eliminar atributos innecesarios** cuanto antes: reduce el tamaño de las tuplas intermedias y evita cargar o mover datos irrelevantes.

- **Reordenar las juntas (`JOIN`)**: el orden en que se aplican las juntas afecta el costo. Explorar distintos órdenes puede reducir drásticamente la cantidad de combinaciones necesarias.

> Estas reglas no cambian el resultado lógico de la consulta, pero sí impactan fuertemente en su **eficiencia**.

### Reglas de Reescritura


Aunque no trabajemos directamente con Álgebra Relacional, muchas optimizaciones pueden analizarse a nivel de las cláusulas SQL. Algunas reglas comunes son:

- **Unificación de predicados `WHERE`**: múltiples condiciones `WHERE` encadenadas pueden agruparse en un solo predicado con operadores `AND`, sin alterar el resultado.

- **Orden de filtros por destructividad**: aplicar primero el filtro más restrictivo (el que descarta más filas) es lógicamente equivalente a aplicar los filtros más suaves de forma incremental, pero suele ser más eficiente.

- **Conmutatividad y asociatividad de `JOIN`**: el orden lógico en que se escriben las juntas no afecta el resultado final, lo que permite al optimizador reordenarlas buscando la forma más rápida de ejecutarlas.

- **Mapeo de productos cartesianos a juntas**: si después de un `CROSS JOIN` se aplica un `WHERE` que relaciona columnas de ambas tablas, el optimizador puede reescribirlo como un `INNER JOIN` más eficiente.

- **Mover selecciones dentro de juntas** (*predicados locales*): si una condición del `WHERE` involucra solo una de las tablas de una junta, puede aplicarse **antes** de la junta para reducir el tamaño de entrada.


## Algoritmos de Resolución de Juntas




Los algoritmos para ejecutar `JOIN` definen cómo combinar eficientemente las tuplas de dos relaciones. A continuación se presentan dos enfoques básicos:

#### Iteración ingenua
- Examina **todos los pares posibles** de tuplas entre las dos relaciones.
- Muy costosa si ambas relaciones son grandes.
- **Costo**: `bloques(A) + tuplas(A) * bloques(B)`

#### Iteración por bloques (páginas)
- Se toma cada bloque de A y se **itera sobre los bloques de B**, buscando coincidencias.
- Más eficiente si una de las relaciones cabe completamente en memoria.
- **Costo**: `bloques(A) + bloques(A) * bloques(B)`

#### Método Hash
- Se construye en **memoria** una **tabla hash** con los valores de los atributos de junta de la **relación más pequeña** (almacenando sus direcciones).
- Luego se **recorre la otra relación**, aplicando la función hash y buscando coincidencias en la tabla construida.
- Es muy eficiente cuando una de las relaciones **cabe completamente en memoria**.
- **Costo**: Es aproximadamente la suma de las páginas/bloques de ambas relaciones.

#### Uso de índices
- Se aprovechan **estructuras de ordenación** ya existentes en el SGBD para acceder directamente a las tuplas relevantes.
- Muy útil cuando solo una **fracción** de la relación secundaria participa en la junta.
- **Costo**: acceso al **índice** + acceso a las **tuplas correspondientes**.

#### Sort-Merge
- Se **ordenan ambas relaciones** por los atributos de junta y luego se recorren secuencialmente en paralelo, uniendo las coincidencias.
- Requiere ordenar si los datos no están ya ordenados.
- **Costo**: costo de ordenación (`B log B`) + **suma de bloques** de ambas relaciones.

